In [1]:
import pandas as pd

In [2]:
data_de_seattle = pd.read_csv('job_data_engineer_Seattle.csv')
data_de_ny = pd.read_csv('job_data_engineer_NYC.csv')
data_de_sf = pd.read_csv('job_data_engineer_sanfrancisco.csv')
data_ds_seattle = pd.read_csv('job_data_scientist_Seattle.csv')
data_ds_ny = pd.read_csv('job_data_scientist_NYC.csv')
data_ds_sf = pd.read_csv('job_data_scientist_sanfrancisco.csv')
data_da_seattle = pd.read_csv('job_data_analyst_Seattle.csv')
data_da_ny = pd.read_csv('job_data_analyst_NYC.csv')
data_da_sf = pd.read_csv('job_data_analyst_sanfrancisco.csv')

data = pd.concat([data_de_seattle, data_de_ny, data_de_sf, data_ds_seattle, data_ds_ny, data_ds_sf, data_da_seattle, data_da_ny, data_da_sf], ignore_index=True)
# Check for duplicates based on URLs
data.drop_duplicates(subset=['role', 'location', 'company'], inplace=True)
print(len(data))

2470


In [3]:
# Create new Column Job Type -> either Data Analyst, Engineer, Business INtelligence, Data Scientist, Machine LEarning, software development
#Assigning the job type to each row in the dataset
import re

patterns = {
    'DE': r'Data\s*Engineer',
    'DA': r'Data\s+.*?\s*(Analy(?:st|sis|tics|tic))\b(?:,)?',
    'BI': r'Business\s*Intelligence|BI',
    'DS': r'Data\s*Scientist|Data\s*Science',
    'SD': r'Software\s*(?:Development|Engineer)',
    'ML': r'Machine\s*Learning|ML',
    'RE': r'Researcher\s'
}

def assign_type(role):
    for job_type, pattern in patterns.items():
        if pd.notnull(role) and pd.notnull(pattern) and re.search(pattern, role, flags=re.IGNORECASE):
            return job_type
    return 'other'

# Apply the function to create the 'type' column
data['type'] = data['role'].apply(assign_type)

In [4]:
# Create new column to see Professional level (Director, Staff, Intern, Principal/Manager)
#Assigning the professional level to each row in the dataset
data['level'] = data['role'].apply(lambda x: 'DI' if 'Director' in x else
                                      ('ST' if 'Staff' in x else
                                      ('IN' if 'Intern' in x or 'Internship' in x else
                                      ('PR' if 'Principal' in x or 'Manager' in x else'normal'))))

In [5]:
# Create new column to see work schedule (on-site or hybrid)
#Assigning the professional level to each row in the dataset
data['schedule'] = data.apply(lambda row: 'on-site' if 'on-site' in str(row['location']).lower() or 'on-site' in str(row['description']).lower()
                              else ('hybrid' if 'hybrid' in str(row['location']).lower() or 'hybrid' in str(row['description']).lower()
                                    else ('remote' if 'remote' in str(row['location']).lower() or 'remote' in str(row['description']).lower()
                                          else 'NaN')), axis=1)

In [6]:
# extract salary range
# Extract salary range using regex and convert to strings
# Extract salary range using regex
salary_range = data['description'].str.extract(r'\$([\d,]+)[^\d\w]*([\d,]+)')

# Assign 'min' and 'max' columns
data['min_salary'] = salary_range[0]
data['max_salary'] = salary_range[1]

# Fill NaN values with 'NaN'
data['min_salary'].fillna('NaN', inplace=True)
data['max_salary'].fillna('NaN', inplace=True)

In [7]:
data.tail()

,url,company,location,role,description,applicants,post_days,type,level,schedule,min_salary,max_salary
3031,https://www.linkedin.com/jobs/view/3845656519/...,FICO,"San Rafael, CA (Remote)",Analytic Consultant I,\n \n About the job\n ...,100.0,30.0,other,normal,remote,"100,00",0
3036,https://www.linkedin.com/jobs/view/3893322149/...,Braintrust,San Francisco Bay Area (Remote),Cyber Risk/ Data Analyst - US Only,\n \n About the job\n ...,65.0,4.0,DA,normal,remote,NaN,NaN
3037,https://www.linkedin.com/jobs/view/3830558462/...,Stripe,"San Francisco, CA (On-site)","Masters, Data Analyst Intern",\n \n About the job\n ...,100.0,60.0,DA,IN,on-site,"137,800","137,800"
3038,https://www.linkedin.com/jobs/view/3895436803/...,Square,"San Francisco, CA (Remote)","Senior Data Analyst, Square Banking",\n \n About the job\n ...,100.0,3.0,DA,normal,remote,"135,20",0
3039,https://www.linkedin.com/jobs/view/3863873275/...,US Tech Solutions,"San Francisco, CA (Remote)","Fraud/Risk Data Analyst (SQL, Snowflake",\n \n About the job\n ...,100.0,21.0,DA,normal,remote,NaN,NaN


In [8]:
import re
import pandas as pd

operational_skills = pd.read_csv('operational.csv')
packages_skills = pd.read_csv('packages.csv')
tech_skills = pd.read_csv('technologies.csv')


# Convert skills lists to lowercase
operational_skills['Skill'] = operational_skills['Skill'].str.lower()
packages_skills['Skill'] = packages_skills['Skill'].str.lower()
tech_skills['Skill'] = tech_skills['Skill'].str.lower()

# Combine all skill sets into a single set
all_skills_set = set(operational_skills['Skill']).union(set(packages_skills['Skill']), set(tech_skills['Skill']))

# Convert the combined set back to a list without duplicates
all_skills_list = list(all_skills_set)

# Create a new DataFrame to store the results
result_data = pd.DataFrame()
data['description'] = data['description'].str.lower()

# Apply the regular expression pattern to each description in the 'description' column
for skill in all_skills_list:
    if len(skill) == 1:
        pattern = r"\b{}\b".format(re.escape(skill))
    else:
        pattern = re.compile(re.escape(skill), flags=re.IGNORECASE)
    result_data[skill] = data['description'].apply(lambda row: 'yes' if re.findall(pattern, str(row)) else 'no')

# Concatenate the original data DataFrame with the result_data DataFrame
data = pd.concat([data, result_data], axis=1)

In [9]:
data.head()

,url,company,location,role,description,applicants,post_days,type,level,schedule,...,presto,m/r,kafka,networkx,clickhouse,jira,c++,power bi,ggplot2,bigquery
0,https://www.linkedin.com/jobs/view/3830178037/...,Google,"Seattle, WA (On-site)","Développeur de solutions terrain III, IA génér...",\n \n about the job\n ...,1.0,30.0,other,normal,on-site,...,no,no,no,no,no,no,no,no,no,no
1,https://www.linkedin.com/jobs/view/3871458999/...,Adobe,"Seattle, WA",Senior ML Training GPU Optimization Engineer,\n \n about the job\n ...,10.0,14.0,ML,normal,NaN,...,no,no,no,no,no,no,yes,no,no,no
2,https://www.linkedin.com/jobs/view/3877281608/...,Amazon Web Services (AWS),"Seattle, WA","Sr. Physical Security Engineer, Data Center De...",\n \n about the job\n ...,0.0,14.0,other,normal,NaN,...,no,no,no,no,no,no,no,no,no,no
3,https://www.linkedin.com/jobs/view/3788649375/...,Google,"Kirkland, WA (On-site)","Principal Engineer, Query Processing, Spanner,...",\n \n about the job\n ...,69.0,30.0,other,PR,on-site,...,no,no,no,no,no,no,no,no,no,no
4,https://www.linkedin.com/jobs/view/3834018551/...,Google,"Seattle, WA (On-site)","Développeur de solutions terrain II, IA généra...",\n \n about the job\n ...,2.0,30.0,other,normal,on-site,...,no,no,no,no,no,no,no,no,no,no


In [10]:
# Assuming your dataframe is named 'df' and the column is named 'Location'
data['city'] = [location.split(',')[0].strip() if ',' in location else 'remote' for location in data['location']]
data['state'] = [location.split(',')[1].strip() if ',' in location else 'remote' for location in data['location']]
#remove hybrid and remote
data['state'] = data['state'].str.replace('(On-site)', '').str.replace('(Hybrid)', '')
#data[['city', 'state']] = data[['city', 'state']].fillna('remote')

In [11]:
data.head()

,url,company,location,role,description,applicants,post_days,type,level,schedule,...,kafka,networkx,clickhouse,jira,c++,power bi,ggplot2,bigquery,city,state
0,https://www.linkedin.com/jobs/view/3830178037/...,Google,"Seattle, WA (On-site)","Développeur de solutions terrain III, IA génér...",\n \n about the job\n ...,1.0,30.0,other,normal,on-site,...,no,no,no,no,no,no,no,no,Seattle,WA
1,https://www.linkedin.com/jobs/view/3871458999/...,Adobe,"Seattle, WA",Senior ML Training GPU Optimization Engineer,\n \n about the job\n ...,10.0,14.0,ML,normal,NaN,...,no,no,no,no,yes,no,no,no,Seattle,WA
2,https://www.linkedin.com/jobs/view/3877281608/...,Amazon Web Services (AWS),"Seattle, WA","Sr. Physical Security Engineer, Data Center De...",\n \n about the job\n ...,0.0,14.0,other,normal,NaN,...,no,no,no,no,no,no,no,no,Seattle,WA
3,https://www.linkedin.com/jobs/view/3788649375/...,Google,"Kirkland, WA (On-site)","Principal Engineer, Query Processing, Spanner,...",\n \n about the job\n ...,69.0,30.0,other,PR,on-site,...,no,no,no,no,no,no,no,no,Kirkland,WA
4,https://www.linkedin.com/jobs/view/3834018551/...,Google,"Seattle, WA (On-site)","Développeur de solutions terrain II, IA généra...",\n \n about the job\n ...,2.0,30.0,other,normal,on-site,...,no,no,no,no,no,no,no,no,Seattle,WA


In [12]:
data['state'] = data['state'].apply(lambda x: 'CA' if 'CA' in x or 'California' in x else x)
data['state'] = data['state'].apply(lambda x: 'NY' if 'NY' in x or 'New York' in x else x)
data['state'] = data['state'].apply(lambda x: 'remote' if 'remote' in x or 'United States' in x else x)
data['state'] = data['state'].apply(lambda x: 'WA' if 'WA' in x or 'Washington' in x else x)

data['city'] = data['city'].apply(lambda x: 'New York' if 'New York' in x else x)
data['city'] = data['city'].apply(lambda x: 'San Francisco' if 'San Francisco' in x else x)

In [18]:
data.to_csv('LI_Data_Jobs.csv', index=False)